In [1]:
!pip install gradio

In [2]:
import gradio as gr
import os
import shutil
import os

import pandas as pd
import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

def zip_files(files):
    print(files)
    for idx, file in enumerate(files):
        shutil.move(file.name, './Upload/'+os.path.split(file.name)[1])
        
           
    os.system("rm -r ./temp")
    os.system("rm -r ./temp2")
    os.system("rm -r ./crop")
    os.system("rm -r ./0")
    os.system("rm -r ./1")
    os.system("rm -r ./2")
    
    os.system("mkdir ./temp2")
    os.system("mkdir ./crop")    
    os.system("mkdir ./temp")
    os.system("mkdir ./0")
    os.system("mkdir ./1")
    os.system("mkdir ./2")
    
    os.system("cp -R ./Upload/* ./temp")
    subdirs = os.listdir('./temp')

    for i in subdirs:
        os.rename('./temp/'+i, './temp/'+i[:-4]+'__'+i[-4:])
    os.system("python ./yolov5/detect.py --weights yolov5x6.pt  --conf 0.5 --save-crop --source ./temp/ --project ./crop/ --name Cygnus --exist-ok")

        
    os.system("cp -R ./crop/Cygnus/crops/bird/* ./temp2")
        
        
    model = tf.keras.models.load_model("model_mobilenet_stage2.h5", compile=False)
    
    
    BATCH_SIZE = 1
    IMG_SIZE = (224, 224)
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator()
    filter_dataset = datagen.flow_from_directory('./temp2/',
                                                 target_size=IMG_SIZE,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=False,
                                                 classes=[''])
    prediction = model.predict(filter_dataset, verbose=1)
    fn = filter_dataset.filenames
    
    
    
    D = {}

    cnt = 0
    fn = filter_dataset.filenames
    prediction1 = np.argmax(prediction[:, :], axis=1)
    prediction2 = np.argmax(prediction[:, :-1], axis=1)
    
    for p1, p2 in zip(prediction1, prediction2):
        D[fn[cnt]] = {'prediction_all': p1, 'prediction_no_other': p2}
        cnt += 1
    
    D = pd.DataFrame(D)
    D = D.T.reset_index()
    D.columns = ['file_name', 'prediction_all', 'prediction_no_other']
    D['crop_num'] = D['file_name'].apply(lambda x: x.split('__')[1]).values
    D['file_name'] = D['file_name'].apply(lambda x: x.split('__')[0]+'.'+x.split('__')[1].split('.')[-1]).values

    E=D.groupby(['file_name'])['prediction_no_other'].mean().astype(int).reset_index()
    E.columns = ['name', 'class']
    E.to_csv(f"submission_v2.csv", sep=';', index = False)        

    

    for i in E.index:
        shutil.copyfile('./Upload/'+E['name'].loc[i], './'+E['class'].loc[i].astype(str)+'/'+E['name'].loc[i])
        
    E['class'][E['class']==0]='Cygnus bewickii'
    E['class'][E['class']==1]='Cygnus cygnus'
    E['class'][E['class']==2]='Cygnus olor'
    return E

demo = gr.Interface(
    zip_files,
    gr.File(file_count="multiple"),
    "text"
)

if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b7e8fd9023bb300001.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


[<tempfile._TemporaryFileWrapper object at 0x7f941c0bf6d0>, <tempfile._TemporaryFileWrapper object at 0x7f941c0bc220>, <tempfile._TemporaryFileWrapper object at 0x7f941c0bfbe0>, <tempfile._TemporaryFileWrapper object at 0x7f941c0bde40>, <tempfile._TemporaryFileWrapper object at 0x7f941c0bdb70>]


detect: weights=['yolov5x6.pt'], source=./temp/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=./crop/, name=Cygnus, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-168-gec2b853 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (Tesla P40, 24446MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
image 1/8 /root/work/Hakaton/2023/CP2023_lebed/test/temp/1227728682__.jpg: 640x448 1 bird, 79.8ms
image 2/8 /root/work/Hakaton/2023/CP2023_lebed/test/temp/1233594995__.jpg: 448x640 1 bird, 80.3ms
image 3/8 /root/work/Hakaton/2023/CP2023_lebed/test/temp/1428357358__.jpg: 384x640 1 bird, 80.6ms
image 4/8 /root/work/Hakaton/2023/CP2023_lebed/test/temp/1428556015__.jpg: 512x640 2 bi

requirements: /root/miniconda3/lib/python3.10/site-packages/requirements.txt not found, check failed.
Found 6 images belonging to 1 classes.
6/6 [==============================] - 6s 22ms/step


/tmp/ipykernel_73668/1016277182.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  E['class'][E['class']==0]='Cygnus bewickii'
